# Requires parts database to already be downloaded and unziped (JLCPCB Database\cache.sqlite3)

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect(r"JLCPCB Database\cache.sqlite3")
cur = conn.cursor()

# Create a new FTS table
cur.execute(
    """
    CREATE VIRTUAL TABLE components_fts
    USING fts5
    (lcsc, category_id, mfr, package, description, datasheet)
"""
)

# Insert data from the components table into the FTS table
cur.execute(
    "SELECT lcsc, category_id, mfr, package, description, datasheet FROM components"
)
rows = cur.fetchall()
insert_stmt = """
    INSERT INTO components_fts (lcsc, category_id, mfr, package, description, datasheet)
    VALUES (?, ?, ?, ?, ?, ?)
"""
cur.executemany(insert_stmt, rows)

# Commit the changes
conn.commit()

# Close the connection
conn.close()

In [ ]:
import sqlite3
import json

# Search terms
search_terms = [
    "SARA-R520M10",
    "SARA-R520",
    "BG950A-GL",
    "GM02SP",
    "L710",
    "HL7810",
    "HL7812",
    "BG95-M3",
    "L710HG",
    "BG95-M6",
    "SKY66431",
    "SARA-R510AWS",
    "SARA-R422",
    "SARA-R422S",
    "SARA-R422M10S",
    "SIM7070G",
    "BG770A-GL",
    "SIM7080G",
    "RYZ024A",
    "ME310G1-W3",
    "ME910G1-W3",
    "TX62-W",
    "TPM540",
    "MA510-GL",
    "TX82-W",
    "EXS62-W",
    "RYZ014A",
    "EXS82-W",
    "BG95-M6",
    "BG95-M5",
    "BG95",
    "BG77",
    "BG95-M1",
    "BG95-M2",
    "ME910C1-P1",
    "GM01Q",
    "HL7800-M",
    "WP7702",
    "SARA-R410M-52B",
    "ME910C1-WW",
    "ML865C1-NA",
    "SARA-R410M-02B",
    "ME910C1-NA",
    "nRF9160",
]

# Connect to the database
conn = sqlite3.connect(r"JLCPCB Database\cache.sqlite3")
cur = conn.cursor()

query = """
    SELECT c.price, c.stock, f.*
    FROM components c
    JOIN components_fts f ON c.lcsc = f.lcsc
    WHERE f.mfr LIKE ? OR f.description LIKE ? OR f.datasheet LIKE ? OR f.lcsc LIKE ?
"""

rows = []
for term in search_terms:
    # Execute the query
    cur.execute(
        query,
        (f"%{term}%",) * 4,
    )

    # Fetch the results and append to rows
    rows.extend(cur.fetchall())

# Sort the rows based on the CoreMark/$ ratio
rows.sort(
    key=lambda row: json.loads(row[0])[0]["price"],
    reverse=True,
)

# Print the results
for i, row in enumerate(rows):
    price_data = json.loads(row[0])
    print(f"{row[4]}")
    print(f"Price: {round(price_data[0]['price'], 2)}USD")
    print(f"Stock: {row[1]}")
    print(f"Package: {row[5]}")
    print(f"Description: {row[6]}")
    print(f"https://jlcpcb.com/partdetail/C{row[2]}")
    print()

# Close the connection
conn.close()